In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
shared_idx, shared_col, collective_frames = {}, {}, {}
for root, dirs, files in os.walk("../third-party/Beijing Air-Quality/PRSA2017_Data_20130301-20170228/"):
    for filename in files:
        dataframe = pd.read_csv("%s/%s" % (root, filename), sep=',', index_col=0)
        # If you have values approximating a cumulative distribution function, then should work well
        dataframe = dataframe.interpolate(method='pchip', limit_direction='both')
        dataframe.index = pd.to_datetime(dataframe[['year', 'month', 'day', 'hour']])
        # Dropping the Wind Direction, no way to fill missing values
        dataframe = dataframe.drop(['wd', 'year', 'month', 'day', 'hour'], axis=1)
        dataframe.station = dataframe.station.str.title().sort_index()
        station_name = list(set(dataframe.station.values))[0]
        dataframe = dataframe.drop(['station'], axis=1)
        # Identifying the shared timestamps and shared columns
        shared_idx = set(dataframe.index) if len(shared_idx) == 0 else shared_idx & set(dataframe.index)
        shared_col = set(dataframe.columns) if len(shared_col) == 0 else shared_col & set(dataframe.columns)
        # Saving the current dataset for later use
        collective_frames[station_name] = dataframe

In [3]:
shared_idx = sorted(shared_idx)
shared_col = sorted(shared_col)

In [4]:
dataframes = []
for key in list(sorted(collective_frames.keys())):
    dataframe = collective_frames[key].loc[shared_idx]
    dataframe = dataframe[shared_col]
    dataframes.append(dataframe.values)
matrix = np.stack(dataframes)

In [5]:
shared_idx = [str(idx) for idx in shared_idx]
shared_col = [str(idx) for idx in shared_col]

In [6]:
np.save("../prepared-data/Beijing Air-Quality (%dx%dx%d)" % matrix.shape, matrix)

In [7]:
with open("../prepared-data/Beijing Air-Quality (%dx%dx%d).AX0" % matrix.shape,'w') as f:
    f.write('\n'.join(list(sorted(collective_frames.keys()))))

In [8]:
with open("../prepared-data/Beijing Air-Quality (%dx%dx%d).AX1" % matrix.shape,'w') as f:
    f.write('\n'.join(shared_idx))

In [9]:
with open("../prepared-data/Beijing Air-Quality (%dx%dx%d).AX2" % matrix.shape,'w') as f:
    f.write('\n'.join(shared_col))